# Classification de texte avec embeddings SpaCy et régression logistique

Ce notebook détaille un pipeline complet de classification de texte utilisant les embeddings pré-entraînés de SpaCy (en_core_web_lg) et la régression logistique. Chaque étape est expliquée, du prétraitement à la génération du fichier de soumission.

---

## Concepts clés

- **Embeddings SpaCy** : chaque texte est représenté par un vecteur dense (300 dimensions) issu d'un modèle pré-entraîné, capturant la sémantique globale du texte.
- **Régression logistique** : modèle linéaire de classification multi-classes, qui prédit la probabilité d'appartenance à chaque classe.
- **Prétraitement linguistique** : nettoyage léger pour améliorer la qualité des features.
- **Train/Test split** : séparation des données pour évaluer la capacité de généralisation du modèle.

---

In [ ]:
# Importation des bibliothèques nécessaires
import json
import re
import pandas as pd
import numpy as np
import spacy
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## 1. Chargement et nettoyage des données

On charge les données d'entraînement depuis un fichier JSON. On remplace les valeurs manquantes dans la colonne 'description' par une chaîne vide pour éviter les erreurs lors du traitement.

In [ ]:
with open('train.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)
df = pd.DataFrame(train_data)
df['description'] = df['description'].fillna("")

## 2. Chargement du modèle SpaCy large (en_core_web_lg)

On utilise le modèle SpaCy large, qui fournit des embeddings de 300 dimensions pour chaque texte. Ces vecteurs sont appris sur de grands corpus et capturent la similarité sémantique entre textes.

In [ ]:
# Assure-toi d'avoir exécuté : python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg')

## 3. Fonction de nettoyage et vectorisation

On définit une fonction qui :
- enlève le HTML et les URLs
- met en minuscules
- passe le texte dans SpaCy pour obtenir le vecteur global (`doc.vector`)

Le vecteur retourné est la moyenne des embeddings des tokens du texte (dimension 300).

In [ ]:
def clean_and_vectorize(text):
    txt = re.sub(r'<[^>]+>', ' ', text)
    txt = re.sub(r'http\S+', ' ', txt)
    txt = txt.lower().strip()
    doc = nlp(txt)
    return doc.vector

## 4. Création de la matrice de features

On applique la fonction à chaque description pour obtenir une matrice $(n_{samples}, 300)$ où chaque ligne est le vecteur du texte. On charge aussi les labels associés à chaque Id.

In [ ]:
vectors = np.vstack(df['description'].apply(clean_and_vectorize).values)
y = pd.read_csv('train_label.csv').set_index('Id').loc[df['Id'], 'Category'].values

## 5. Séparation train/validation

On sépare les données en un ensemble d'entraînement (80%) et de validation (20%) pour évaluer la capacité de généralisation du modèle. On stratifie pour conserver la proportion des classes.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    vectors, y, test_size=0.2, stratify=y, random_state=42
)

## 6. Entraînement du modèle de régression logistique

La régression logistique est un modèle linéaire de classification multi-classes. Elle apprend un hyperplan de séparation dans l'espace des embeddings.

Mathématiquement, pour chaque classe $k$ :
$$ P(y=k|x) = \frac{\exp(w_k^T x + b_k)}{\sum_j \exp(w_j^T x + b_j)} $$
où $x$ est le vecteur du texte, $w_k$ et $b_k$ sont les paramètres appris pour la classe $k$.

In [ ]:
clf = LogisticRegression(max_iter=1000, n_jobs=-1)
clf.fit(X_train, y_train)

## 7. Évaluation du modèle

On prédit les classes sur l'ensemble de validation et on affiche un rapport de classification (précision, rappel, F1-score).

In [ ]:
y_pred = clf.predict(X_val)
print(classification_report(y_val, y_pred))

## 8. Prédiction sur les données de test

On applique le même prétraitement et la même vectorisation aux textes de test, puis on prédit la catégorie pour chaque texte.

In [ ]:
with open('test.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)
test_df = pd.DataFrame(test_data)
test_vectors = np.vstack(test_df['description'].apply(clean_and_vectorize).values)
preds = clf.predict(test_vectors)

## 9. Génération du fichier de soumission

On prépare le fichier de soumission au format attendu, associant chaque Id de test à la catégorie prédite.

In [ ]:
template = pd.read_csv('template_submissions.csv')
template['Category'] = template['Id'].map(dict(zip(test_df['Id'], preds)))
template.to_csv('submission_spacy_lg.csv', index=False)
print("Saved submission as 'submission_spacy_lg.csv'")

---

## Concepts mathématiques et conclusion

- **Embeddings SpaCy** : chaque texte est représenté par un vecteur dense de 300 dimensions, qui capture la sémantique globale du texte.
- **Régression logistique** : le modèle apprend à séparer les classes dans l'espace vectoriel des textes, en maximisant la probabilité des bonnes classes.
- **Classification** : la sortie du modèle est la classe avec la probabilité la plus élevée.

Ce pipeline montre comment passer de textes bruts à des prédictions de classes avec un modèle linéaire, en utilisant des représentations vectorielles avancées.